In [ ]:
per_entities = ["John Dalton", "Mary", "David", "Sarah"]
loc_entities = ["New York", "London", "Paris", "Tokyo"]
prod_entities = ["apple", "iPhone 14", "iPhone",  "PlayStation", "Tesla Model S", "Nike Air Max"]
cw_entities = ["Hamlet", "The Great Gatsby", "Mona Lisa", "Starry Night"]
grp_entities = ["family", "friends", "colleagues", "students"]
corp_entities = ["Ford", "apple", "Toyota", "Samsung", "Coca Cola"]

# write named entities to text files
with open("PER.txt", "w") as f:
    f.write("\n".join(per_entities))
with open("LOC.txt", "w") as f:
    f.write("\n".join(loc_entities))
with open("PROD.txt", "w") as f:
    f.write("\n".join(prod_entities))
with open("CW.txt", "w") as f:
    f.write("\n".join(cw_entities))
with open("GRP.txt", "w") as f:
    f.write("\n".join(grp_entities))
with open("CORP.txt", "w") as f:
    f.write("\n".join(corp_entities))

In [ ]:
import os
import numpy as np

def create_one_hot_encoding(string):
    # Define the named entities
    string = string.lower()
    named_entities = ["PER", "LOC", "PROD", "CW", "GRP", "CORP"]
    
    # Define the tags
    tags = ["B", "I"]
    
    # Create a dictionary to map named entities to tags
    entity_to_tag = {}
    for i, entity in enumerate(named_entities):
        entity_to_tag[entity] = tags[i % len(tags)]
    
    # Load the text files
    entity_sets = {}
    for entity in named_entities:
        file_path = os.path.join("/content/", "{}.txt".format(entity))
        entity_sets[entity] = set(open(file_path, "r", encoding="utf-8").read().lower().split())
    
    # Create the one-hot encoding for each word
    one_hot_encoding = []
    str_idx = 0
    for word in string.split():
        encoded_word = np.zeros(len(named_entities) * len(tags))
        for i, entity in enumerate(named_entities):
            tag = entity_to_tag[entity]
            if word in entity_sets[entity]:
                if str_idx == 0:
                    encoded_word[i * len(tags)] = 1
                    print("if")
                else:
                  prev_tag = one_hot_encoding[str_idx-1][i * len(tags)] == 1 or one_hot_encoding[str_idx-1][i * len(tags) + 1] == 1
                  print("else")
                  if not prev_tag:
                      encoded_word[i * len(tags)] = 1
                  else:
                      encoded_word[(i * len(tags)) + (prev_tag % len(tags))] = 1
            # else:
            #     encoded_word[(i * len(tags)) + len(tags) - 1] = 1
        str_idx += 1
        one_hot_encoding.append(encoded_word)
    
    return np.array(one_hot_encoding)

In [ ]:
%%time
a = create_one_hot_encoding("please give me the pen")
print("  PR PR LC LC PD PD CW CW GR GR CRP CRP")
print(a)

else
  PR PR LC LC PD PD CW CW GR GR CRP CRP
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
CPU times: user 5.35 ms, sys: 23 µs, total: 5.37 ms
Wall time: 6.43 ms


In [ ]:
entity_sets = {}
named_entities = ["PER", "LOC", "PROD", "CW", "GRP", "CORP"]

for entity in named_entities:
    file_path = os.path.join("/content/", "{}.txt".format(entity))
    entity_sets[entity] = set(open(file_path, "r", encoding="utf-8").read().lower().split())

print(entity_sets)

{'PER': {'john', 'david', 'mary', 'sarah', 'dalton'}, 'LOC': {'tokyo', 'paris', 'london', 'york', 'new'}, 'PROD': {'max', 'tesla', '14', 'playstation', 'model', 'apple', 'iphone', 'air', 's', 'nike'}, 'CW': {'mona', 'night', 'gatsby', 'great', 'starry', 'lisa', 'hamlet', 'the'}, 'GRP': {'students', 'friends', 'family', 'colleagues'}, 'CORP': {'coca', 'ford', 'samsung', 'apple', 'cola', 'toyota'}}


#LSTM


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
data_path = "//content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/train2022withPos.csv"

In [ ]:
data = pd.read_csv(data_path, encoding= 'utf-8')
# filling the first column that determines which sentence each word belongs to.
data.fillna(method = 'ffill', inplace = True)
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,২০১৮,RDF,O
1,Sentence: 1,এর,PPR,O
2,Sentence: 1,সেরা,PPR,O
3,Sentence: 1,বর্ণানুক্রমিকভাবে,AMN,O
4,Sentence: 1,তালিকাভুক্ত,JJ,O


In [ ]:
def join_a_sentence(sentence_number):

    """
    Args.:
          sentence_number: sentence number we want to join and return. 
          
    Returns:
          The joined sentence.
    """
    
    sentence_number = str(sentence_number)
    the_sentence_words_list = list(data[data['Sentence #'] == 'Sentence: {}'.format(sentence_number)]['Word'])
    
    return ' '.join(the_sentence_words_list)

In [ ]:
# Data Shape
data.shape

(191509, 4)

In [4]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.entity_tags = ["PER", "LOC", "CW", "CORP", "GRP", "PROD"]
        self.tag_encoding = {"O": 0, "B-PER": 1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4,
                             "B-CW": 5, "I-CW": 6, "B-CORP": 7, "I-CORP": 8, 
                             "B-GRP": 9, "I-GRP": 10, "B-PROD": 11, "I-PROD": 12}

    def insert(self, word, entity_type):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.entity_type = entity_type

    def search(self, sentence):
        encoding = [0] * len(sentence)
        for i in range(len(sentence)):
            node = self.root
            for j in range(i, len(sentence)):
                char = sentence[j]
                if char not in node.children:
                    break
                node = node.children[char]
                if node.is_end_of_word:
                    entity_type = node.entity_type
                    entity_length = j - i + 1
                    start_index = i
                    end_index = j
                    for k in range(start_index, end_index + 1):
                        if k == start_index:
                            encoding[k] = self.tag_encoding["B-" + entity_type]
                        else:
                            encoding[k] = self.tag_encoding["I-" + entity_type]
                    break
        one_hot_encoding = [[0] * 13 for i in range(len(encoding))]
        for i, tag in enumerate(encoding):
            one_hot_encoding[i][tag] = 1
        return one_hot_encoding

In [5]:
trie = Trie()

# Insert some named entities into the Trie
trie.insert("John", "PER")
trie.insert("New York", "LOC")
trie.insert("iPhone", "PROD")
trie.insert("Microsoft", "CORP")

# Search for named entities in a sentence
sentence = "John lives in New York and owns an iPhone made by Apple, not Microsoft."
encoding = trie.search(sentence)
print(encoding)

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0,

In [12]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.label = None

class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.entities = set()

    def insert(self, word, label):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.label = label
        self.entities.add(label)

    def search(self, sentence):
        encoding = []
        for word in sentence.split():
            word_encoding = [0] * (len(self.entities) * 2 + 1)
            node = self.root
            for char in word:
                if char not in node.children:
                    break
                node = node.children[char]
            else:
                if node.label is not None:
                    label_index = sorted(list(self.entities)).index(node.label)
                    word_encoding[label_index * 2] = 1  # Begin
                    for i in range(1, len(word)):
                        word_encoding[label_index * 2 + 1] = 1  # Inside
                    encoding.append(word_encoding)
                    continue
            encoding.append(word_encoding)
        return encoding

In [14]:
trie = Trie()

# Insert some named entities into the Trie
trie.insert("John", "PER")
trie.insert("New York", "LOC")
trie.insert("iPhone", "PROD")
trie.insert("Microsoft", "CORP")

# Search for named entities in a sentence
sentence = "John lives in New York and owns an iPhone made by Apple, not Microsoft."
encoding = trie.search(sentence)
print(*encoding, sep = "\n")

[0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
